In [1]:
from pylab import *
import matplotlib.pyplot as plt
import random
import numpy as np
from queryParser import *
import psycopg2
import time

# input="SELECT SUM(l.lo_extendedprice * l.lo_discount) AS revenue FROM lineorder l , date d WHERE l.lo_orderdate = d.d_datekey AND d.d_year = 1993 AND l.lo_discount BETWEEN 1 AND 3 AND l.lo_quantity < 25;"


tablesWithSel={}
sortedTables=[]
FACT="lineorder"
def FanOut(input,cursor):
    state=[]
    solution={}
    query="select count(*) from "+FACT
    print("query",query)
    cursor.execute(query)
    factCar=cursor.fetchall()[0][0]
    # calculer les cardinalités des tables
    joinedTables ,parsed_query , alias=queryParser(input)
    print(alias)
    joinedTables.remove(FACT)
    print(joinedTables)
    tables=get_tableWithSelectivity(parsed_query)
    print(tables)
    
    for key in tables.keys():
        json={'select': {'value': {'count': '*'}},'from':  [{'value': 'lineorder', 'name': 'l'},{'left join': {'name':key,'value':alias[key]}, 'on': {'eq': get_joinedClause(alias[key])}}]}
        if len(tables[key])==1:
            json['where']=tables[key][0]
        else:
            json['where']={'and':tables[key]}
        query=format(json)
        print("query",query)
        cursor.execute(query)
        tablesWithSel[alias[key]]=cursor.fetchall()[0][0]/factCar
    print("tablewithsel",tablesWithSel)
    if(len(joinedTables)>len(tablesWithSel)):
        for t in joinedTables:
            if t not in tablesWithSel:
                query="select count(*) from lineorder l left join "+t+" as "+get_key(t,alias)+" on "+get_joinedClause(t)[0]+" = "+get_joinedClause(t)[1]
                print("query",query)
                cursor.execute(query)
                tablesWithSel[t]=cursor.fetchall()[0][0]/factCar
                
    
    sortedTables={k: v for k, v in sorted(tablesWithSel.items(), key=lambda item: item[1])}
    state=list(sortedTables.keys())
    state.insert(1,FACT)
    print(state)
    solution["query"]=listToQuery(state,get_indice(state),parsed_query)
    #solution["runTime"]=get_runTime(solution["query"],cursor)
    solution["estimatedCost"]=get_cost(solution["query"],cursor)
    #time.sleep(5)
    #solution["energy"]=get_energy(solution["query"],cursor)
    return solution



In [2]:
conn,cursor = connect_bdd("ssb")
input="SELECT d.d_year,s.s_city,p.p_brand1,SUM(l.lo_revenue - l.lo_supplycost) AS profit  FROM date d,customer c,supplier s,part p,lineorder l WHERE l.lo_custkey = c.c_custkey AND l.lo_suppkey = s.s_suppkey  AND l.lo_partkey = p.p_partkey AND l.lo_orderdate = d.d_datekey  AND c.c_region = 'AMERICA' AND s.s_nation = 'UNITED STATES' AND (d.d_year = 1997 OR d.d_year = 1998)  AND p.p_category = 'MFGR#14'  GROUP BY  d.d_year, s.s_city, p.p_brand1 ORDER BY d.d_year,s.s_city,p.p_brand1 ;"
#input="SELECT c_city,s_city,d_year,SUM(lo_revenue) AS revenue FROM customer c,lineorder l,supplier s,date d WHERE  l.lo_custkey = c.c_custkey AND l.lo_suppkey = s.s_suppkey AND l.lo_orderdate = d.d_datekey AND (c.c_city='UNITED KI1' OR c.c_city='UNITED KI5') AND (s.s_city='UNITED KI1' OR s.s_city='UNITED KI5') AND d.d_yearmonth = 'Dec1997'  GROUP BY  c.c_city, s.s_city, d.d_year ORDER BY  d.d_year ASC,  revenue DESC;"
solution = FanOut(input,cursor)
print(solution)


query select count(*) from lineorder
{'d': 'date', 'c': 'customer', 's': 'supplier', 'p': 'part', 'l': 'lineorder'}
['date', 'customer', 'supplier', 'part']
{'c': [{'eq': ['c.c_region', {'literal': 'AMERICA'}]}], 's': [{'eq': ['s.s_nation', {'literal': 'UNITED STATES'}]}], 'd': [{'or': [{'eq': ['d.d_year', 1997]}, {'eq': ['d.d_year', 1998]}]}], 'p': [{'eq': ['p.p_category', {'literal': 'MFGR#14'}]}]}
query SELECT COUNT(*) FROM lineorder AS l LEFT JOIN customer AS c ON l.lo_custkey = c.c_custkey WHERE c.c_region = 'AMERICA'
query SELECT COUNT(*) FROM lineorder AS l LEFT JOIN supplier AS s ON l.lo_suppkey = s.s_suppkey WHERE s.s_nation = 'UNITED STATES'
query SELECT COUNT(*) FROM lineorder AS l LEFT JOIN date AS d ON l.lo_orderdate = d.d_datekey WHERE d.d_year = 1997 OR d.d_year = 1998
query SELECT COUNT(*) FROM lineorder AS l LEFT JOIN part AS p ON l.lo_partkey = p.p_partkey WHERE p.p_category = 'MFGR#14'
tablewithsel {'customer': 0.19913371048169495, 'supplier': 0.04022882770501556, 'd

In [8]:
conn,cursor = connect_bdd("ssb")
input="SELECT d.d_year,s.s_city,p.p_brand1,SUM(l.lo_revenue - l.lo_supplycost) AS profit  FROM date d,customer c,supplier s,part p,lineorder l WHERE l.lo_custkey = c.c_custkey AND l.lo_suppkey = s.s_suppkey  AND l.lo_partkey = p.p_partkey AND l.lo_orderdate = d.d_datekey  AND c.c_region = 'AMERICA' AND s.s_nation = 'UNITED STATES' AND (d.d_year = 1997 OR d.d_year = 1998)  AND p.p_category = 'MFGR#14'  GROUP BY  d.d_year, s.s_city, p.p_brand1 ORDER BY d.d_year,s.s_city,p.p_brand1 ;"
#input="SELECT c_city,s_city,d_year,SUM(lo_revenue) AS revenue FROM customer c,lineorder l,supplier s,date d WHERE  l.lo_custkey = c.c_custkey AND l.lo_suppkey = s.s_suppkey AND l.lo_orderdate = d.d_datekey AND (c.c_city='UNITED KI1' OR c.c_city='UNITED KI5') AND (s.s_city='UNITED KI1' OR s.s_city='UNITED KI5') AND d.d_yearmonth = 'Dec1997'  GROUP BY  c.c_city, s.s_city, d.d_year ORDER BY  d.d_year ASC,  revenue DESC;"
solution = get_cost(input,cursor)
print(solution)

3442532.71


In [10]:
t={"a":0}
del(t["a"])
t

{}